In [1]:
import numpy as np
from numba import jit

In [2]:
to_group_e_min = 0.4
to_group_e_max = 0.5

In [3]:
from_group_e_min = 0.6
from_group_e_max = 0.7

In [4]:
alpha = ((12. - 1.) / (12. + 1.))**2

In [5]:
from_es = np.linspace(from_group_e_min, from_group_e_max)
to_es = np.linspace(to_group_e_min, to_group_e_max)

In [6]:
@jit
def numba_my_function(ef, et):
    scale = np.log(from_group_e_max) - np.log(from_group_e_min)
    def flux(e):
        return 1 / e / scale
    def sigma(e_from, e_to, sig=5):
        if e_to < alpha * e_from:
            return 0
        elif e_to > e_from:
            return 0
        else:
            return sig
    return sigma(ef, et, 5) / ef * flux(ef) / (1 - alpha)

In [8]:
%%timeit
contributions = []

num_bins = 5000

from_energies = np.linspace(from_group_e_min, from_group_e_max, num_bins)
to_energies = np.linspace(to_group_e_min, to_group_e_max, num_bins)

delta_e_from = from_energies[1] - from_energies[0]
delta_e_to = to_energies[1] - to_energies[0]

ef = from_group_e_min

answer = 0

for ef in from_energies:
    for et in to_energies:
        c = 0.5 * (my_function(ef, et) + my_function(ef+delta_e_from, et+delta_e_to))
        c *= delta_e_from * delta_e_to
        answer += c

print(answer)

0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
4min 14s ± 3.46 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
contributions = []

num_bins = 5000

from_energies = np.linspace(from_group_e_min, from_group_e_max, num_bins)
to_energies = np.linspace(to_group_e_min, to_group_e_max, num_bins)

delta_e_from = from_energies[1] - from_energies[0]
delta_e_to = to_energies[1] - to_energies[0]

ef = from_group_e_min

answer = 0

for ef in from_energies:
    for et in to_energies:
        c = 0.5 * (numba_my_function(ef, et) + numba_my_function(ef+delta_e_from, et+delta_e_to))
        c *= delta_e_from * delta_e_to
        answer += c
        
print(answer)

0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
0.991960307295
35.8 s ± 711 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
contributions = []

num_bins = 10000

from_energies = np.linspace(from_group_e_min, from_group_e_max, num_bins)
to_energies = np.linspace(to_group_e_min, to_group_e_max, num_bins)

delta_e_from = from_energies[1] - from_energies[0]
delta_e_to = to_energies[1] - to_energies[0]

ef = from_group_e_min

answer = 0

for ef in from_energies:
    for et in to_energies:
        c = 0.5 * (numba_my_function(ef, et) + numba_my_function(ef+delta_e_from, et+delta_e_to))
        c *= delta_e_from * delta_e_to
        answer += c
        
print(answer)

0.991691406151


In [18]:
@jit
def unit_contribution(ef, d_ef, et, d_et):
    c = 0.5 * (numba_my_function(ef, et) + numba_my_function(ef+d_ef, et+d_et))
    c *= d_ef * d_et
    return c

In [45]:
%time sum([unit_contribution(ef, delta_e_from, et, delta_e_to) \
 for ef in from_energies for et in to_energies if unit_contribution(ef, delta_e_from, et, delta_e_to) > 0.0])

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs
CPU times: user 1min 18s, sys: 631 ms, total: 1min 18s
Wall time: 1min 18s
CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11.9 µs


In [ ]:
sum([unit_contribution(ef, delta_e_from, et, delta_e_to) for ef in from_energies for et in to_energies])